The first step will be to __trim/clean our raw Illumina data__.

In [ ]:
!mkdir trimming

In [ ]:
cd trimming

Prepare a text file specifying the samples to be processed including the format and location of the reads. 

The below command expects the Illumina data to be present in 2 fastq files (forward and reverse reads) per sample in a directory `./raw_reads/`. It expects the files to be named 'sampleID-marker', followed by '\_1' or '\_2' to identify the forward/reverse read file respectively. sampleID must corresponds to the first column in the file `Sample_accessions.tsv`, marker is either '12S' or 'CytB'. 

Read file names, for example:
```
./raw_reads/Bassenthwaite_01-12S_1.fastq.gz
./raw_reads/Bassenthwaite_01-12S_2.fastq.gz
./raw_reads/Bassenthwaite_01-CytB_1.fastq.gz
./raw_reads/Bassenthwaite_01-CytB_2.fastq.gz
./raw_reads/Bassenthwaite_02-12S_1.fastq.gz
./raw_reads/Bassenthwaite_02-12S_2.fastq.gz
./raw_reads/Bassenthwaite_02-CytB_1.fastq.gz
./raw_reads/Bassenthwaite_02-CytB_2.fastq.gz
./raw_reads/Bassenthwaite_03-12S_1.fastq.gz
./raw_reads/Bassenthwaite_03-12S_2.fastq.gz
```


In [ ]:
%%bash

for a in $(cat ../../supplementary_data/Sample_accessions.tsv | grep "12S" | cut -f 1 | grep "SampleID" -v)
do
    R1=$(ls -1 ../../raw_reads/$a-12S_* | grep "_1.fastq")
    R2=$(ls -1 ../../raw_reads/$a-12S_* | grep "_2.fastq")

    echo -e "$a\tfastq\t$R1\t$R2"
done > Querymap.txt

The resulting file should look e.g. like below:

In [50]:
!head Querymap.txt

Bassenthwaite_01-nc	fasta	../chimera_detection/Bassenthwaite_01/Bassenthwaite_01-nonchimeras.fasta
Bassenthwaite_02-nc	fasta	../chimera_detection/Bassenthwaite_02/Bassenthwaite_02-nonchimeras.fasta
Bassenthwaite_03-nc	fasta	../chimera_detection/Bassenthwaite_03/Bassenthwaite_03-nonchimeras.fasta
Bassenthwaite_04-nc	fasta	../chimera_detection/Bassenthwaite_04/Bassenthwaite_04-nonchimeras.fasta
Bassenthwaite_05-nc	fasta	../chimera_detection/Bassenthwaite_05/Bassenthwaite_05-nonchimeras.fasta
Bassenthwaite_shore-01-nc	fasta	../chimera_detection/Bassenthwaite_shore-01/Bassenthwaite_shore-01-nonchimeras.fasta
Derwent_01-nc	fasta	../chimera_detection/Derwent_01/Derwent_01-nonchimeras.fasta
Derwent_02-nc	fasta	../chimera_detection/Derwent_02/Derwent_02-nonchimeras.fasta
Derwent_03-nc	fasta	../chimera_detection/Derwent_03/Derwent_03-nonchimeras.fasta
Derwent_04-nc	fasta	../chimera_detection/Derwent_04/Derwent_04-nonchimeras.fasta


The 12S amplicon sequenced here is only 106bp long. Readlength used in the MiSeq run was 2x300bp. Our reads are thus longer than our amplicon and we so expect to find primer/adapter sequences in our reads that need to be removed as part of the raw data processing. 

Specifically, forward reads are expected to contain the reverse complement of the reverse primer plus the reverse Illumina adapter (FA501 - FA508), and reverse reads will contain reverse complements of the forward primers and adapters (RB701 - RB712).

The expected sequences have been prepared in the below file `12S_adapters_rc.fasta` and will be used in the trimming algorithm.

In [51]:
!head ../12S-adapters_rc.fasta

>FA501_rc
GGGGTATCTAATCCCAGTCCAATTACCATACGTACGATGTGTAGATCTCGGTGGTCGCCGTATCATT
>FA502_rc
GGGGTATCTAATCCCAGTCCAATTACCATACAGATAGTGTGTAGATCTCGGTGGTCGCCGTATCATT
>FA503_rc
GGGGTATCTAATCCCAGTCCAATTACCATAACTCGCTAGTGTAGATCTCGGTGGTCGCCGTATCATT
>FA504_rc
GGGGTATCTAATCCCAGTCCAATTACCATAACACGCAGGTGTAGATCTCGGTGGTCGCCGTATCATT
>FA505_rc
GGGGTATCTAATCCCAGTCCAATTACCATACTCGATGAGTGTAGATCTCGGTGGTCGCCGTATCATT


__Raw data trimming, removal of adapter sequences and merging of reads__ using the `metaBEAT` pipeline.

In [ ]:
%%bash

metaBEAT.py \
-Q Querymap.txt \
--trim_qual 30 \
--trim_adapter ../12S-adapters_rc.fasta \
--trim_minlength 90 \
--merge \
--product_length 110 \
-n 5 -v &> log


In [ ]:
cd ../

Some stats on the read counts before/after trimming, merging etc. are summarized for you in `read_stats.csv`.

Next stage of the processing is __chimera detection__ and removal of putative chimeric sequences. We'll do that using `uchime` as implemented in `vsearch`.

In [ ]:
!mkdir chimera_detection

In [ ]:
cd chimera_detection

Convert reference database from Genbank to fasta format to be used in chimera detection.

Prepare Refmap file, i.e. text file that specifies the location and the format of the reference to be used.

The reference sequences in Genbank format should already be present in the `12S` directory: `custom_extended_12S.gb`.

In [ ]:
%%bash

#Write REFmap
for file in $(ls -1 ../* | grep "gb$")
do
    echo -e "$file\tgb"
done > REFmap.txt

In [52]:
!cat REFmap.txt

../custom_extended_12S.gb	gb


In [ ]:
%%bash

metaBEAT.py \
-R REFmap.txt \
-f

This will produce `refs.fasta`.

Now run chimera detection.

In [ ]:
%%bash


for a in $(cut -f 1 ../trimming/Querymap.txt)
do
    if [ -s ../trimming/$a/$a\_trimmed.fasta ]
    then
        echo -e "\n### Detecting chimeras in $a ###\n"
        mkdir $a
        cd $a
        vsearch --uchime_ref ../../trimming/$a/$a\_trimmed.fasta --db ../refs.fasta \
        --nonchimeras $a-nonchimeras.fasta --chimeras $a-chimeras.fasta &> log
        cd ..

    else
        echo -e "$a is empty"
    fi
done




In [ ]:
cd ..

Last step is __taxonomic assignment of reads based on a BLAST - LCA approach__ using the metaBEAT pipeline.

In [ ]:
!mkdir non-chimeras

In [ ]:
cd non-chimeras/

Prepare Querymap and Refmap txt files.

In [ ]:
%%bash

#Querymap
for a in $(ls -l ../chimera_detection/ | grep "^d" | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
    echo -e "$a-nc\tfasta\t../chimera_detection/$a/$a-nonchimeras.fasta"
done > Querymap.txt

#REFmap
#Write REFmap
for file in $(ls -1 ../* | grep "gb$")
do
    echo -e "$file\tgb"
done > REFmap.txt

Sequence clustering and taxonomic assignment via metaBEAT.

In [ ]:
%%bash

metaBEAT.py \
-Q Querymap.txt \
-R REFmap.txt \
--cluster --clust_match 1 --clust_cov 3 \
--blast --min_ident 1 \
-m 12S -n 5 \
-E -v \
-o 12S-trim_30-merged-nonchimeras-cluster_1c3-blast-min_ident_1.0 &> log

Final result of taxonomic assignment can be found in the table `12S-trim_30-merged-nonchimeras-cluster_1c3-blast-min_ident_1.0.tsv` (see also [here](https://github.com/HullUni-bioinformatics/Haenfling_et_al_2016/blob/master/supplementary_data/assignment_results/12S-trim_30-nonchimeras-cluster_1c3-blast-min_ident_1.0.tsv)). 

metaBEAT also produced the final result in [BIOM](http://biom-format.org/) format (`12S-trim_30-merged-nonchimeras-cluster_1c3-blast-min_ident_1.0.biom`), which can be analyzed with a number of tools and visually explored e.g. using [Phinch](http://phinch.org/).